In [1]:

import tensorflow as tf 
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.linear_model import SGDClassifier
sess=tf.InteractiveSession()




#==================EXTRACTING AND ANALYSING DATA FROM .mat FILES===============
"""
X and Y Components of Training and Testing Data
"""
train_data = scipy.io.loadmat('train_32x32.mat')['X']
train_labels = scipy.io.loadmat('train_32x32.mat')['y']
test_data = scipy.io.loadmat('test_32x32.mat')['X']
test_labels = scipy.io.loadmat('test_32x32.mat')['y']

train_data = train_data.astype('float32') / 128.0 - 1
test_data = test_data.astype('float32') / 128.0 - 1


"""
Converting Labels to One Hot Encoding and Image Matrix to favourable dimensions
"""
def reformat(data):
    xtrain = []
    trainLen = data.shape[3]
    for x in range(trainLen):
        xtrain.append(data[:,:,:,x])
    xtrain = np.asarray(xtrain)
    
    return xtrain
train_data = reformat(train_data)
test_data = reformat(test_data)
train_labels=tf.one_hot(indices=train_labels,depth=10)
train_labels=train_labels.eval().reshape(73257,10)
test_labels=tf.one_hot(indices=test_labels,depth=10)
test_labels=test_labels.eval().reshape(26032,10)






In [14]:
#============================================================================

#==================BUILDING THE CNN==========================================
"""
Various Hyperparameters required for training the CNN.
"""
image_size = 32
width = 32
height = 32
channels = 3

n_labels = 10
patch = 5
depth = 8
hidden = 128
dropout = 0.9375

num_epochs=30
batch = 128
learning_rate = 0.00001

"""
Constructing the placeholders and variables in the TensorFlow Graph
"""
#Training Dataset
tf_train_dataset = tf.placeholder(tf.float32, shape=(None, width, height, channels))
#Training Labels
tf_train_labels = tf.placeholder(tf.float32, shape=(None, n_labels))
#Testing Dataset
tf_test_dataset = tf.constant(test_data)

#   Layer 1: (5, 5, 3, 16)
layer1_weights = tf.Variable(tf.truncated_normal([patch, patch, channels, depth], stddev=0.1))
layer1_biases = tf.Variable(tf.constant(0.1, shape=[depth]))

#   Layer 2: (5, 5, 16, 16)
layer2_weights = tf.Variable(tf.truncated_normal([patch, patch, depth, depth], stddev=0.1))
layer2_biases = tf.Variable(tf.constant(0.1, shape=[depth]))
#   Layer 3: (5, 5, 16, 16)
layer3_weights = tf.Variable(tf.truncated_normal([patch, patch, depth, depth], stddev=0.1))
layer3_biases = tf.Variable(tf.constant(0.1, shape=[depth]))

#   Layer 4: (1024, 128)
layer4_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, hidden], stddev=0.1))
layer4_biases = tf.Variable(tf.constant(0.1, shape=[hidden]))

#   Layer 5: (128, 10)
layer5_weights = tf.Variable(tf.truncated_normal([hidden, n_labels], stddev=0.1))
layer5_biases = tf.Variable(tf.constant(0.1, shape=[n_labels]))

dropout = tf.placeholder(tf.float32)

def model(data):
    #   Convolution 1 and RELU
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden1 = tf.nn.relu(conv1 + layer1_biases)
    #   Max Pool
    hidden2 = tf.nn.max_pool(hidden1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    #   Convolution 2 and RELU
    conv2 = tf.nn.conv2d(hidden2, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden3 = tf.nn.relu(conv2 + layer2_biases)
    #   Max Pool
    hidden4 = tf.nn.max_pool(hidden3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    #   Convolution 3 and RELU
    conv3 = tf.nn.conv2d(hidden4, layer3_weights, [1, 1, 1, 1], padding='SAME')
    #hidden5 = tf.nn.relu(conv3 + layer3_biases)
    #   Max Pool
    #hidden6 = tf.nn.max_pool(hidden5, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    shape = conv3.get_shape().as_list()

    reshape = tf.reshape(conv3, [-1, shape[1] * shape[2] * shape[3]])
    hidden5 = tf.nn.relu(tf.matmul(reshape, layer4_weights) + layer4_biases)
    #   Dropout
    dropout_layer = tf.nn.dropout(hidden5, dropout)
    
    return tf.matmul(dropout_layer, layer5_weights) + layer5_biases

logits = model(tf_train_dataset)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
train_prediction = tf.nn.softmax(logits)
test_prediction = tf.nn.softmax(model(tf_test_dataset))

#============================================================================



In [15]:
#==================TRAINING AND TESTING THE MODEL============================
"""
Accuracy function defined similar to the one taught in the Udacity Deep Learning Course
Returns percentage of correct predictions by verifying with Labels
"""
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])



tf.global_variables_initializer().run()
print('Initialized')


for epoch in range(num_epochs):
    sum_cost = 0.
    sum_accu = 0.
    num_steps=int(73257/batch)
    for step in range(num_steps):
    #   Constucting the batch from the data set
        offset = step * batch
        batch_data = train_data[offset:(offset + batch), :, :, :]
        batch_labels = train_labels[offset:(offset + batch), :]
        #   Dictionary to be fed to TensorFlow Session
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, dropout: 0.93}
        _, c, predictions = sess.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        #   Calculating the Accuracy of the predictions
        accu = accuracy(predictions, batch_labels)
        if (step % 100 == 0):
            print('Minibatch loss at step %d: %f' % (step, c))
            print('Minibatch accuracy: %.1f%%' % accu)
        sum_cost += c 
        sum_accu += accu 
    avg_cost=sum_cost/ num_steps
    avg_accu=sum_accu/ num_steps
    print('loss at epoch %d: %f' % (epoch, avg_cost))
    print('accuracy: %.1f%%' % avg_accu)
print("Optimization Finished!")    





Initialized
Minibatch loss at step 0: 2.267957
Minibatch accuracy: 15.6%
Minibatch loss at step 100: 2.112753
Minibatch accuracy: 9.4%
Minibatch loss at step 200: 2.014346
Minibatch accuracy: 16.4%
Minibatch loss at step 300: 1.973937
Minibatch accuracy: 21.9%
Minibatch loss at step 400: 1.992520
Minibatch accuracy: 21.1%
Minibatch loss at step 500: 2.026473
Minibatch accuracy: 16.4%
loss at epoch 0: 2.061017
accuracy: 17.9%
Minibatch loss at step 0: 2.038059
Minibatch accuracy: 23.4%
Minibatch loss at step 100: 1.989894
Minibatch accuracy: 16.4%
Minibatch loss at step 200: 1.962062
Minibatch accuracy: 17.2%
Minibatch loss at step 300: 1.914169
Minibatch accuracy: 19.5%
Minibatch loss at step 400: 2.014383
Minibatch accuracy: 21.1%
Minibatch loss at step 500: 2.000835
Minibatch accuracy: 18.8%
loss at epoch 1: 1.992553
accuracy: 21.1%
Minibatch loss at step 0: 1.980010
Minibatch accuracy: 22.7%
Minibatch loss at step 100: 1.996945
Minibatch accuracy: 18.8%
Minibatch loss at step 200: 1

KeyboardInterrupt: 

In [19]:
    print("Average Accuracy : ", (average / num_steps))
    print("END OF TRAINING")
    average = 0
    for step in range(num_steps):
    #   Constucting the batch from the data set
    offset = (step * batch) % (26032 - batch)
    batch_data = test_data[offset:(offset + batch), :, :, :]
    batch_labels = test_labels[offset:(offset + batch), :]
    #   Dictionary to be fed to TensorFlow Session
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, dropout: 0.93}
    _, l, predictions = sess.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    #   Calculating the Accuracy of the predictions
    accu = accuracy(predictions, batch_labels)
    if (step % 50 == 0):
        print('Minibatch loss at step %d: %f' % (step, l))
        print('Minibatch accuracy: %.1f%%' % accu)
    average += accu
    print( "Average Accuracy : ", (average / num_steps))
    print ("END OF TESTING")
    #============================================================================

IndentationError: expected an indented block (<ipython-input-19-cbc88d1cb8ad>, line 6)

In [ ]:
    """for i in range(10000):
    batch = train_data.next_batch(50)
    train_accuracy = accuracy.eval(feed_dict={
        tf_train_dataset:batch[0], tf_train_labels: batch[1], keep_prob: 1.0})
    if (i % 50 == 0):

        print("step %d, training accuracy %.4f"%(i, train_accuracy))
    average += train_accuracy
    optimizer.run(feed_dict={tf_train_dataset: batch[0], tf_train_labels: batch[1], keep_prob: 0.5})
    average += accu
    print("test accuracy %.4f"%accuracy.eval(feed_dict={
    x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0}))
    print("cross entropy %.3f"%cross_entropy.eval(feed_dict={
    x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0})) """  